#### Encoding the context of the domain dataset for Taiwan Indigenous Peoples
Wen-Hsin Yang, May 17, 2022

## 台灣原住民族 領域資料之專用資料(共通資料): 編列代碼
1. 身分別(台灣原住民)
2. 族別((台灣原住民族名稱)
3. 部落別(部落名稱、部落名稱_名族名稱)

### 讀取台灣原住民族相關資料內容
1. 原住民身分
2. 核定台灣原住民族
3. 核定部落、區域及居民所屬族別)
4. 核定台灣原住民族各族語言及方言(根據原住民族語言能力認證測驗之分類)

In [1]:
import pandas as pd

fileName = r'專用資料_input.xlsx'
with pd.ExcelFile(fileName) as xlsxFile:
    df_id = pd.read_excel(xlsxFile, sheet_name='身分別', header=0)
    df_tip = pd.read_excel(xlsxFile, sheet_name='族別', header=0)
    df_tribe = pd.read_excel(xlsxFile, sheet_name='部落別', header=0)
    df_language = pd.read_excel(xlsxFile, sheet_name='族語_方言別', header=0)

#### 1. 編列原住民族身分別代碼 (1~ 2)

In [2]:
df_id['身分別_代碼'] = '0' # add on one column in the df_id dataframe
df_id.dropna(inplace=True)
for i in range(df_id.shape[0]):
    df_id['身分別_代碼'].iloc[i] = format(df_id['編號'].iloc[i].astype(int), '01d')
df_id

c:\Users\whyan\Anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,編號,身分,身分別_代碼
0,0,非原住民,0
1,1,平地原住民,1
2,2,山地原住民,2


#### 2. 編列原住民族 核定民族別 代碼 (01~16)

#### * 建立字典 { 族名:代碼 } 

In [3]:
df_tip['族別_代碼'] = '0' # add on one column in the df_tip dataframe
df_tip.dropna(inplace=True)
dict_tip = dict() # create the dictionary for the key-value pair of the 'tip'-'tip_id'
for i in range(df_tip.shape[0]):
    df_tip['族別_代碼'].iloc[i] = format(df_tip['編號'].iloc[i].astype(int), '02d')
    dict_tip[df_tip['族別'].iloc[i].strip()] = df_tip['族別_代碼'].iloc[i]
df_tip['族別_代碼'].iloc[i] = '00'
dict_tip[df_tip['族別'].iloc[i].strip()] = '00' # 因為'未登記'放在所有核准族別之後，但是編列特殊代碼'00'
df_tip.dropna(inplace=True)
dict_tip

c:\Users\whyan\Anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


{'阿美族': '01',
 '泰雅族': '02',
 '排灣族': '03',
 '布農族': '04',
 '卑南族': '05',
 '魯凱族': '06',
 '鄒族': '07',
 '賽夏族': '08',
 '雅美族': '09',
 '邵族': '10',
 '噶瑪蘭族': '11',
 '太魯閣族': '12',
 '撒奇萊雅族': '13',
 '賽德克族': '14',
 '拉阿魯哇族': '15',
 '卡那卡那富族': '16',
 '未登記': '00'}

#### 3. 編列族語及方言代碼( tribal language、dialect)

In [4]:
language = list() # 有幾個民族族語
_lang = ''
language_idx = 0
dialect_idx = 0
df_language['方言別_代碼'] = '0' # 加入方言別_代碼 欄位
df_language['族語+方言_代碼'] = '0' # 加入族語+方言_代碼 欄位
for i in range(df_language.shape[0]):
    _temp = df_language['族語'].iloc[i]
    # 考慮 excel工作表內，族語-方言 對照表格，族語欄使用合併儲存格方式表達
    # 先標示 一個民族 (e.g., 阿美族語)，之後下一列如沒有載明(空白)，則代表還是'阿美族語'
    # 如下一列載明 另一個民族 (e.g., 泰雅族語，則代表從此列開始是'泰雅族語'相關之方言
    if pd.isnull(_temp):
        _temp = _lang
        df_language['族語'].iloc[i] = _temp
        dialect_idx += 1 # 計算屬於這個族語的方言數量(也是方言的編號)
    else:
        language_idx += 1 # 計算屬於特定族語的數量
        _lang = _temp
        language.append(_temp)
        dialect_idx = 1 # # 計算屬於這個族語的方言數量(也是方言的編號)
    
    df_language['方言別_代碼'].iloc[i] = format(dialect_idx, '02d') # 放入方言別 代碼欄
    df_language['族語+方言_代碼'].iloc[i] = format(language_idx, '02d')+format(dialect_idx, '02d') # 放入 族語+方言 代碼欄
    
# encoding for the tribal language (16族語言)
# 族語 list，透過enumerate編號，建成dictionary (族語：代碼)
dict_language = dict()
for count, value in enumerate(language, start=1):
    dict_language[value] = format(count, '02d') # 邊
    print(count, value)
# dictionary (族語：編號)，轉成
_langCode = pd.DataFrame(list(dict_language.items()), columns=['族語', '族語_代碼'])

# 使用原始表格內用的欄位名稱(方言，有標示是採用'原住民族語言能力認證測驗')，因此改成用第幾個欄位 access
col_name = df_language.columns[1]
df_language[['族語', col_name, '方言別_代碼', '族語+方言_代碼']]

1 阿美族語
2 泰雅族語
3 排灣族語
4 布農族語
5 卑南族語
6 魯凱族語
7 鄒族語
8 賽夏族語
9 雅美族語
10 邵族語
11 噶瑪蘭族語
12 太魯閣族語
13 撒奇萊雅族語
14 賽德克族語
15 拉阿魯哇族語
16 卡那卡那富族語


,族語,方言_原住民族語言能力認證測驗,方言別_代碼,族語+方言_代碼
0,阿美族語,南勢阿美語,01,0101
1,阿美族語,秀姑巒阿美語,02,0102
2,阿美族語,海岸阿美語,03,0103
3,阿美族語,馬蘭阿美語,04,0104
4,阿美族語,恆春阿美語,05,0105
5,泰雅族語,賽考利克泰雅語,01,0201
6,泰雅族語,四季泰雅語,02,0202
7,泰雅族語,澤敖利泰雅語,03,0203
8,泰雅族語,宜蘭澤敖利泰雅語,04,0204
9,泰雅族語,汶水泰雅語,05,0205


#### 4. 編列原住民族 核定部落代碼(0001 ~ 0735)、核定部落+族別代碼('0001 ~ 0375' + 族別1 '01 ~ 16' + 族別2 '01 ~ 16' + 族別3 '01 ~ 16' +...)

#### conduct parsing for the 行政區域(省市縣市鄉鎮區) of the 內政部戶政資料代碼表

In [5]:
#import pandas as pd
fileName = '.\行政區域_20220503.xlsx'
with pd.ExcelFile(fileName) as xlsxFile:
    df = pd.read_excel(xlsxFile, header=None)

df['代碼'] = '0' # 加入行政區域_代碼 欄位
df['行政區域'] = '0'
dict_areacode = {}
for i in range(df.shape[0]):
    area = df[0].iloc[i] # 讀入 戶政資料代碼:省市縣市鄉鎮市區代碼
    # extract 代碼
    code_area = area.split('=')
    code = code_area[0].strip()
    area = code_area[1].strip()
    df['代碼'].iloc[i] = code 
    df['行政區域'].iloc[i] = area 
    #print('{} {} # {} {}'.format(i, code_area, code, area))
    # add an item into the dictionary of the {區域:代碼}
    dict_areacode[area] = code
   
writer = pd.ExcelWriter('.\行政區域代碼_20220503.xlsx')
df[['行政區域', '代碼']].to_excel(writer, sheet_name='代碼', index=False)
writer.save()
#print(dict_areacode)
dict_areacode

{'中華民國內政部': '00000000',
 '臺灣省': '10000000',
 '臺灣省兵役處': '10000006',
 '臺灣省臺北縣': '10001000',
 '臺灣省臺北縣民政局': '10001006',
 '臺灣省臺北縣板橋市': '10001012',
 '臺灣省臺北縣三重市': '10001022',
 '臺灣省臺北縣中和市': '10001032',
 '臺灣省臺北縣永和市': '10001040',
 '臺灣省臺北縣新莊市': '10001050',
 '臺灣省臺北縣新店市': '10001060',
 '臺灣省臺北縣樹林市': '10001070',
 '臺灣省臺北縣樹林鎮': '1000107A',
 '臺灣省臺北縣鶯歌鎮': '10001080',
 '臺灣省臺北縣三峽鎮': '10001090',
 '臺灣省臺北縣淡水鎮': '10001100',
 '臺灣省臺北縣汐止市': '10001110',
 '臺灣省臺北縣汐止鎮': '1000111A',
 '臺灣省臺北縣瑞芳鎮': '10001120',
 '臺灣省臺北縣土城市': '10001130',
 '臺灣省臺北縣蘆洲市': '10001140',
 '臺灣省臺北縣蘆洲鄉': '1000114A',
 '臺灣省臺北縣五股鄉': '10001150',
 '臺灣省臺北縣泰山鄉': '10001160',
 '臺灣省臺北縣林口鄉': '10001170',
 '臺灣省臺北縣深坑鄉': '10001180',
 '臺灣省臺北縣石碇鄉': '10001190',
 '臺灣省臺北縣坪林鄉': '10001200',
 '臺灣省臺北縣三芝鄉': '10001210',
 '臺灣省臺北縣石門鄉': '10001220',
 '臺灣省臺北縣八里鄉': '10001230',
 '臺灣省臺北縣平溪鄉': '10001240',
 '臺灣省臺北縣雙溪鄉': '10001250',
 '臺灣省臺北縣貢寮鄉': '10001260',
 '臺灣省臺北縣金山鄉': '10001270',
 '臺灣省臺北縣萬里鄉': '10001280',
 '臺灣省臺北縣烏來鄉': '10001290',
 '臺灣省宜蘭縣': '10002000',
 '臺灣省宜蘭縣民政處': '10002006',
 '臺

### 編列部落代碼(根據行政區域：省市縣市、鄉鎮市區等兩個層級，對應使用到 內政部戶政資料代碼 (5/17/2022)
1. 處理 臺東縣 臺東市/卑南鄉 卡地布部落(#453)：編列兩個部落代碼，分別是　台東縣臺東市(8碼)+流水號 台東縣卑南鄉(8碼)+流水號)
   * 因為跨越兩個'鄉鎮市區'，所以讀入核定部落清單時，'卡地布部落'會分列兩筆資料(隸屬臺東市、卑南鄉)，但是編號相同(#453)
2. 處理 屏東縣 獅子鄉 新路部落(#721)：編列一個部落代碼 屏東縣獅子鄉(8碼)+流水號
   * 因為因為跨越兩個村里(楓林村、丹路村)，但是'鄉鎮市區'相同(獅子鄉)，所以讀入核定部落清單時，'新路部落'只列一筆資料(隸屬獅子鄉)，編號(#721)

In [6]:
_tribe = df_tribe[['編號', '縣市', '鄉鎮市區', '部落名稱', '族別']]
_tribe['部落別_代碼'] = '0'
_tribe.dropna(inplace=True)
dict_city = {'臺北市':'臺北市', '高雄市':'高雄市', '新北市':'新北市', '臺中市':'臺中市', '臺南市':'臺南市', '桃園市':'桃園市', 
            '台北市':'臺北市', '台中市':'臺中市', '台南市':'臺南市'}
base_area = ''
base_county = '' # base for 省市縣市
dict_parsedCounty = {}
seqNo = 0
for i in range(_tribe.shape[0]):
    # 檢查 是 直轄市 或 省 (省市縣市)
    _area_1 = _tribe['縣市'].iloc[i].strip()
    if _area_1 in dict_city:
        _area = dict_city[_area_1]
    else:
        # 目前只有台灣省，沒有福建省，所以直接加上台灣省
        _area = '臺灣省' + _area_1
    
    if _area != base_county:
        '''
        create county_dictionary
        point to the count_dictionary
        '''
        base_county = _area
        seqNo_parsedTownship = {}
        dict_parsedCounty[base_county] = seqNo_parsedTownship
    
    '''
    retrieve 鄉鎮市區
    create 鄉鎮市區 in the county_dictionary
        if not in (new):
            initiate 鄉鎮市區 (流水號 default 0)
        seqNo = (retreive 流水號 of 鄉鎮市區) + 1
   
    # The following process is revised in accoding to the three lines pseudo code below.
    # 執行  行政區域 完整使用到鄉鎮區層級 (省市縣市鄉鎮市區)
    # 取代  加入各縣市核定部落的流水號，擴編 代碼 (3碼)
    # 產生 部落代碼 (根據area code + seqNo)
    '''
    seqNo = seqNo_parsedTownship.setdefault(_tribe['鄉鎮市區'].iloc[i].strip(), 0) + 1
    seqNo_parsedTownship[_tribe['鄉鎮市區'].iloc[i].strip()] = seqNo

     # 行政區域 完整使用到鄉鎮區層級 (省市縣市鄉鎮市區)
    _area = _area + _tribe['鄉鎮市區'].iloc[i].strip()
    _code = dict_areacode.get(_area, '10000000') # 查表：內政部戶政資料代碼；當找不到時，default設定為'臺灣省'代碼
    _tribe['部落別_代碼'].iloc[i] = _code #dict_areacode.get(_area, '10000000') # 查表：內政部戶政資料代碼；當找不到時，default設定為'臺灣省'代碼

    '''
    加入各縣市核定部落的流水號，擴編 代碼 (3碼)
    '''  
    _tribe['部落別_代碼'].iloc[i] = _code + format(seqNo, '03d')
 
_tribe

c:\Users\whyan\Anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,編號,縣市,鄉鎮市區,部落名稱,族別,部落別_代碼
0,1,新北市,烏來區,桶壁部落,泰雅族,65000290001
1,2,新北市,烏來區,烏來部落,泰雅族,65000290002
2,3,新北市,烏來區,哪哮部落,泰雅族,65000290003
3,4,新北市,烏來區,德拉楠部落,泰雅族,65000290004
4,5,桃園市,復興區,比雅山(角板山)部落,泰雅族,68000130001
...,...,...,...,...,...,...
731,731,屏東縣,獅子鄉,中心崙部落,排灣族,10013320014
732,732,屏東縣,獅子鄉,內獅部落,排灣族,10013320015
733,733,屏東縣,滿州鄉,里德部落,排灣族,10013240001
734,734,屏東縣,滿州鄉,分水嶺部落,排灣族,10013240002


#### * 查族名：代碼 字典，提供編列 部落_族別_代碼

In [7]:
_tribe['部落+族別_代碼'] = '0'
_tribe.dropna(inplace=True)
for i in range(_tribe.shape[0]):
    resident_tribe = _tribe['族別'].iloc[i]
    # 調整輸入'族名'內容
    resident_tribe.strip() # 族名前後有空白
    resident_tribe.replace(' ', '') # 族名字間有空白
    resident = resident_tribe.split('、') # 不同族名間用'、'區隔
    print(resident_tribe, '#', resident)
    _code = list()
    for j in range(len(resident)):
        _resident = resident[j].strip()
        print(_resident, '*', dict_tip[_resident])
        _code.append(dict_tip[_resident])
    code = "".join(_code) # 依序組合(cascade)所有族名代碼
    print(code)
    _tribe['部落+族別_代碼'].iloc[i] =  _tribe['部落別_代碼'].iloc[i] + code + 'AA' # 放入代碼欄，結束碼用'AA'
_tribe

泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族'

c:\Users\whyan\Anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
賽夏族 # ['賽夏族']
賽夏族 * 08
08
賽夏族 # ['賽夏族']
賽夏族 * 08
08
賽夏族 # ['賽夏族']
賽夏族 * 08
08
泰雅族 # ['泰雅族']
泰雅族 * 02
02
賽夏族 # ['賽夏族']
賽夏族 * 08
08
賽夏族 # ['賽夏族']
賽夏族 * 08
08
賽夏族 # ['賽夏族']
賽夏族 * 08
08
賽夏族 # ['賽夏族']
賽夏族 * 08
08
賽夏族 # ['賽夏族']
賽夏族 * 08
08
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
賽夏族 # ['賽夏族']
賽夏族 * 08
08
賽夏族 # ['賽夏族']
賽夏族 * 08
08
賽夏族 # ['賽夏族']
賽夏族 * 08
08
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰雅族 * 02
02
泰雅族 # ['泰雅族']
泰

,編號,縣市,鄉鎮市區,部落名稱,族別,部落別_代碼,部落+族別_代碼
0,1,新北市,烏來區,桶壁部落,泰雅族,65000290001,6500029000102AA
1,2,新北市,烏來區,烏來部落,泰雅族,65000290002,6500029000202AA
2,3,新北市,烏來區,哪哮部落,泰雅族,65000290003,6500029000302AA
3,4,新北市,烏來區,德拉楠部落,泰雅族,65000290004,6500029000402AA
4,5,桃園市,復興區,比雅山(角板山)部落,泰雅族,68000130001,6800013000102AA
...,...,...,...,...,...,...,...
731,731,屏東縣,獅子鄉,中心崙部落,排灣族,10013320014,1001332001403AA
732,732,屏東縣,獅子鄉,內獅部落,排灣族,10013320015,1001332001503AA
733,733,屏東縣,滿州鄉,里德部落,排灣族,10013240001,1001324000103AA
734,734,屏東縣,滿州鄉,分水嶺部落,排灣族,10013240002,1001324000203AA


#### 5. 最後將所有編列的代碼，寫到 '專用資料_代表' 表內
1. 原住民身分別代碼 工作表
2. 原住民族族別代碼 工作表
3. 原住民族核定部落(含族別)代碼 工作表
4. 原住民族族語別代碼 工作表
5. 原住民族族語別(含方言)代碼 工作表

In [8]:
writer = pd.ExcelWriter('專用資料_代碼.xlsx')
df_id.to_excel(writer, sheet_name='身分別_代碼', index=False)
df_tip.to_excel(writer, sheet_name='族別_代碼', index=False)
_tribe.to_excel(writer, sheet_name='部落+族別_代碼', index=False)
_langCode.to_excel(writer, sheet_name='族語_代碼', index=False)
#df_language[['族語', col_name, '方言別_代碼', '族語+方言_代碼']].to_excel(writer, sheet_name='族語+方言_代碼', index=False)
df_language[['族語', df_language.columns[1], '方言別_代碼', '族語+方言_代碼']].to_excel(writer, sheet_name='族語+方言_代碼', index=False)
writer.save()

## The following is kept for backup purpose.

### 保留
#### 編列部落代碼(根據行政區域：省市縣市、鄉鎮市區等兩個層級，對應使用到 內政部戶政資料代碼 
1. 不處理 臺東縣 臺東市/卑南鄉 卡地布部落(#453)：編列兩個部落代碼，分別是　台東縣臺東市(8碼)+流水號 台東縣卑南鄉(8碼)+流水號)
   * 因為跨越兩個'鄉鎮市區'，所以讀入核定部落清單時，'卡地布部落'會分列兩筆資料(隸屬臺東市、卑南鄉)，但是編號相同(#453)
2. 處理 屏東縣 獅子鄉 新路部落(#721)：編列一個部落代碼 屏東縣獅子鄉(8碼)+流水號
   * 因為因為跨越兩個村里(楓林村、丹路村)，但是'鄉鎮市區'相同(獅子鄉)，所以讀入核定部落清單時，'新路部落'只列一筆資料(隸屬獅子鄉)，編號(#721)

In [ ]:
_tribe = df_tribe[['編號', '縣市', '鄉鎮市區', '部落名稱', '族別']]
_tribe['部落別_代碼'] = '0'
_tribe.dropna(inplace=True)
dict_city = {'臺北市':'臺北市', '高雄市':'高雄市', '新北市':'新北市', '臺中市':'臺中市', '臺南市':'臺南市', '桃園市':'桃園市', 
            '台北市':'臺北市', '台中市':'臺中市', '台南市':'臺南市'}
base_area = ''
base_county = ''
seqNo = 0
for i in range(_tribe.shape[0]):
    # 檢查 是 直轄市 或 省 (省市縣市)
    _area_1 = _tribe['縣市'].iloc[i].strip()
    if _area_1 in dict_city:
        _area = dict_city[_area_1]
    else:
        # 目前只有台灣省，沒有福建省，所以直接加上台灣省
        _area = '臺灣省' + _area_1
    # 行政區域 完整使用到鄉鎮區層級 (省市縣市鄉鎮市區)
    _area = _area + _tribe['鄉鎮市區'].iloc[i].strip()
    _code = dict_areacode.get(_area, '10000000') # 查表：內政部戶政資料代碼；當找不到時，default設定為'臺灣省'代碼
    _tribe['部落別_代碼'].iloc[i] = _code #dict_areacode.get(_area, '10000000') # 查表：內政部戶政資料代碼；當找不到時，default設定為'臺灣省'代碼
    '''
    加入各縣市核定部落的流水號，擴編 代碼 (3碼)
    '''
    if _code == base_area:
        seqNo += 1
    else:
        base_area = _code
        seqNo = 1
    
    _tribe['部落別_代碼'].iloc[i] = _code + format(seqNo, '03d')

_tribe